In [ ]:
from openai import OpenAI 
#from api_key import api_key
from ipynb.fs.full.DoctorAppointmentGit import get_doctor_availability,get_doctor,tool

import json

GPT_MODEL = 'gpt-4.1'

openai_api_key = ""


# openapi client 
client = OpenAI(api_key = openai_api_key)

# define role 
initial_prompt = {'role':"system",
                  "content":"""
                                1. You are a helpful assistant designed to book appointments with doctors
                                2. You need to take all info that is relevant to booking
                                3. Ask about their symptoms and preferred doctors and availabity 
                                4. Give them appointment details in a reciept form 
                                5. Answer in in precise manner dont give long answers or questions
                                6. do not send any irrelevant info cause health care is a serious topic dont generate names of deaseas and doctors by your own 

"""}

##handle history 
messages = [initial_prompt]

#color
green = '\033[92m'
red = '\033[91m'
reset = '\033[0m'


#function user input and Ai response 




def handle_response(user_input):
    global messages
    messages.append({'role':'user', "content":user_input})

    response = client.chat.completions.create(
        model = GPT_MODEL,
        messages = messages,
        tools = tool,
        tool_choice="auto")
    
    response_message = response.choices[0].message
    messages.append(response_message)
    print(f'{green}AI: {response_message.content}{reset}')
    #print(response_message)

# tool functionality 
    tool_calls = response_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            tool_call_id = tool_call.id 
            tool_function_name = tool_call.function.name 
            tool_query  = json.loads(tool_call.function.arguments)
            Q = True

            if tool_function_name == 'get_doctors':
                results  = get_doctors(tool_query['query'])
            elif tool_function_name == 'get_doctor_availability':
                results = get_doctor_availability(tool_query['doctor_name'])
            else:
                results = 'error'
                Q = False
            
            if Q:
                messages.append({
                    "role": "tool", 
                    "tool_call_id": tool_call_id, 
                    "name": tool_function_name, 
                    "content": results
                })
            else: 
                print(f"Error: function {tool_function_name} does not exist")

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )

        messages.append(model_response_with_function_call.choices[0].message)
        print(f"{green}AI: {model_response_with_function_call.choices[0].message.content}{reset}")






#intial question

print(f"{green}AI: Hello! I can help you book an appointment with a doctor. Can you please tell me what symptoms you're experiencing?{reset}")

# the flow of converation

while True:
    user_input = input('YOU: ')
    handle_response(user_input) 

AI: Hello! I can help you book an appointment with a doctor. Can you please tell me what symptoms you're experiencing?


YOU:  neck pain


AI: Could you please provide your preferred doctor or specialization and your availability for the appointment?


YOU:  10th AUgust


AI: Thank you. Do you have a preferred doctor or medical specialization for your neck pain appointment?


YOU:  no


AI: None
Error: function get_doctor does not exist


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_lMTaGCTbpSyFy6vYboZpdIuL", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}